In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from IPython.display import display, HTML
import xlrd
import os
import glob
import xgboost as xgb
from datetime import datetime, date
from datetime import timedelta
from dateutil import tz
import json
import requests
import json
import sys

In [2]:
glob.glob("./Downloads/*.csv")

['./Downloads/Loads.csv', './Downloads/cities.csv']

In [3]:
%%time
df = pd.read_csv('./Downloads/Loads.csv', low_memory = False)

CPU times: user 968 ms, sys: 121 ms, total: 1.09 s
Wall time: 1.09 s


In [4]:
df.shape

(65535, 91)

In [5]:
appointmentTimeBoolMap = ~df['Appointment Time'].isna()

In [6]:
np.sum(appointmentTimeBoolMap)

65399

In [7]:
df = df.loc[appointmentTimeBoolMap, :]

In [8]:
df.shape

(65399, 91)

In [9]:
# display(df['Appointment Time'])

In [10]:
print(df.shape)

(65399, 91)


In [11]:
datetime.strptime("05/30/2019 09:45 PM", '%m/%d/%Y %I:%M %p')

datetime.datetime(2019, 5, 30, 21, 45)

In [12]:
edtTimeZone = tz.gettz('US/Eastern')
mdtTimeZone = tz.gettz('US/Mountain')
cestTimeZone = tz.gettz('Europe/Paris')
pdtTimeZone = tz.gettz('US/Pacific')
# for f in df['Appointment Time']:
def getWithTimeZone(f):
    to_zone = tz.gettz('US/Central')
    if f[-3:] == "EDT":
        edt = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
        edt = edt.replace(tzinfo = edtTimeZone)
        f = edt.astimezone(to_zone)
    elif f[-3:] == "MDT":
        mdt = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
        mdt = mdt.replace(tzinfo = mdtTimeZone)
        f = mdt.astimezone(to_zone)
    elif f[-4:] == "CEST":
        cest = datetime.strptime(f[:-5], '%m/%d/%Y %I:%M %p')
        cest = cest.replace(tzinfo = cestTimeZone)
        f = cest.astimezone(to_zone)
    elif f[-3:] == "PDT":
        pdt = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
        pdt = pdt.replace(tzinfo = pdtTimeZone)
        f = pdt.astimezone(to_zone)
    else:
        f = datetime.strptime(f[:-4], '%m/%d/%Y %I:%M %p')
    return pd.to_datetime(f)
        
df.loc[:, 'Appointment Time'] = (df.loc[:, 'Appointment Time'].apply(lambda x: getWithTimeZone(x)))
display(df.head())

,Load Number,PRO Number,Tracking Number,Shipper,Carrier,3pl,Carrier SCAC,Customer,Stop Name,Stop Type,...,ActualDelivery-ETA 24 hours before delivery,ActualDelivery-ETA 12 hours before delivery,ActualDelivery-ETA 8 hours before delivery,ActualDelivery-ETA 4 hours before delivery,ActualDelivery-ETA 2 hours before delivery,ETA 4 hours before delivery,ETA 2 hours before delivery,ETA 4 hours before appointment,ETA 2 hours before appointment,Test Load
0,4300580413,NaN,NaN,Smithfield Foods,Stevens Transport,NaN,STVV,NaN,CU0020001852-KROGER CO-LOUISVILLE,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
1,4300580413,NaN,NaN,Smithfield Foods,Stevens Transport,NaN,STVV,NaN,"SP5087-Cloverleaf-Benson, NC",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
2,4300579679,NaN,NaN,Smithfield Foods,Heyl Truck Lines,NaN,HEYL,NaN,CU0020000407-COSTCO 1036 AIRDRIE DRY,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
3,4300579679,NaN,NaN,Smithfield Foods,Heyl Truck Lines,NaN,HEYL,NaN,"SP1233-SFD-Monmouth,IL",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
4,4300579366,NaN,NaN,Smithfield Foods,Heyl Truck Lines,NaN,HEYL,NaN,CU0020000407-COSTCO 1036 AIRDRIE DRY,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO


In [13]:
# print(df['Appointment Time'])

In [14]:
strtTime = pd.to_datetime("2019-05-30", format = '%Y-%m-%d').date()
endTime = pd.to_datetime("2019-06-08", format = '%Y-%m-%d').date()
df = df[df['Appointment Time'].transform(lambda x: x.date() > strtTime and x.date() < endTime)]
display(df.shape)

(463, 91)

In [15]:
print(df.loc[:, 'Load Number'].isnull().sum())

0


In [16]:
stopboolmap = df.groupby('Load Number')['Stop Type'].transform(lambda x: len(x) == 2)
df = df.loc[stopboolmap, :]
display(df['Load Number'].shape)

(168,)

In [17]:
starttim = pd.to_datetime("2019-05-30", format = '%Y-%m-%d').date()
endtim = pd.to_datetime("2019-06-01", format = '%Y-%m-%d').date()
fd = df[df['Appointment Time'].transform(lambda x: x.date() > starttim and x.date() < endtim)]
x = fd['Truck Number'].shape[0]
fd.loc[:, 'Truck Number'] = np.arange(1, x + 1)
fd.loc[:, 'Truck Number'] = fd.groupby('Load Number')['Truck Number'].transform('first')

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
display(df['Stop Type'].shape)

(168,)

In [19]:
display(df.shape)

(168, 91)

In [20]:
display(len(fd['Truck Number'].unique()))

57

In [21]:
display(df)

,Load Number,PRO Number,Tracking Number,Shipper,Carrier,3pl,Carrier SCAC,Customer,Stop Name,Stop Type,...,ActualDelivery-ETA 24 hours before delivery,ActualDelivery-ETA 12 hours before delivery,ActualDelivery-ETA 8 hours before delivery,ActualDelivery-ETA 4 hours before delivery,ActualDelivery-ETA 2 hours before delivery,ETA 4 hours before delivery,ETA 2 hours before delivery,ETA 4 hours before appointment,ETA 2 hours before appointment,Test Load
39,4300575947,NaN,NaN,Smithfield Foods,Raider Express Inc.,NaN,RDXO,NaN,CU0010001351-CHICKEN E FOODSERVICE INC,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
40,4300575947,NaN,NaN,Smithfield Foods,Raider Express Inc.,NaN,RDXO,NaN,"SP1264-SFD-Cudahy, WI",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
44,4300582209,NaN,NaN,Smithfield Foods,Potter Transport Inc.,NaN,PTGB,NaN,"SP5179-Lineage-Edwardsville, KS",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
45,4300582209,NaN,NaN,Smithfield Foods,Potter Transport Inc.,NaN,PTGB,NaN,SU0000002228-THE DELI SOURCE INC,Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
46,4300572971,NaN,NaN,Smithfield Foods,Ryder SCOS,NaN,RYDE,NaN,"SP1232-SFD-Sioux Falls, SD",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
47,4300572971,NaN,NaN,Smithfield Foods,Ryder SCOS,NaN,RYDE,NaN,SU0001001351-NEBRASKA BEEF LTD,Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
48,4300582207,NaN,NaN,Smithfield Foods,Potter Transport Inc.,NaN,PTGB,NaN,"SP5179-Lineage-Edwardsville, KS",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
49,4300582207,NaN,NaN,Smithfield Foods,Potter Transport Inc.,NaN,PTGB,NaN,SU0000002228-THE DELI SOURCE INC,Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
50,4300576712,NaN,NaN,Smithfield Foods,GUNTREN TRUCKING,NaN,GTNK,NaN,"SP5280-SFD-Greenfield, IN DC",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
51,4300576712,NaN,NaN,Smithfield Foods,GUNTREN TRUCKING,NaN,GTNK,NaN,"SP1263-Turpak Sioux City, IA",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO


In [22]:
print(df['Carrier'])

39                   Raider Express Inc.
40                   Raider Express Inc.
44                 Potter Transport Inc.
45                 Potter Transport Inc.
46                            Ryder SCOS
47                            Ryder SCOS
48                 Potter Transport Inc.
49                 Potter Transport Inc.
50                      GUNTREN TRUCKING
51                      GUNTREN TRUCKING
52                      GUNTREN TRUCKING
53                      GUNTREN TRUCKING
54                      GUNTREN TRUCKING
55                      GUNTREN TRUCKING
56                      GUNTREN TRUCKING
57                      GUNTREN TRUCKING
58                      GUNTREN TRUCKING
59                      GUNTREN TRUCKING
60                      GUNTREN TRUCKING
61                      GUNTREN TRUCKING
71                      Larsen Logistics
72                      Larsen Logistics
79            Hirschbach Motor Lines Inc
80            Hirschbach Motor Lines Inc
81              

In [23]:
df.loc[(df['Carrier'] != "Schuster Co."), 'Truck Number'] = np.nan

In [24]:
df.loc[(df['Carrier'] != "Schuster Co."), 'Carrier'] = np.nan

In [25]:
display(df.head())

,Load Number,PRO Number,Tracking Number,Shipper,Carrier,3pl,Carrier SCAC,Customer,Stop Name,Stop Type,...,ActualDelivery-ETA 24 hours before delivery,ActualDelivery-ETA 12 hours before delivery,ActualDelivery-ETA 8 hours before delivery,ActualDelivery-ETA 4 hours before delivery,ActualDelivery-ETA 2 hours before delivery,ETA 4 hours before delivery,ETA 2 hours before delivery,ETA 4 hours before appointment,ETA 2 hours before appointment,Test Load
39,4300575947,NaN,NaN,Smithfield Foods,NaN,NaN,RDXO,NaN,CU0010001351-CHICKEN E FOODSERVICE INC,Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
40,4300575947,NaN,NaN,Smithfield Foods,NaN,NaN,RDXO,NaN,"SP1264-SFD-Cudahy, WI",Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
44,4300582209,NaN,NaN,Smithfield Foods,NaN,NaN,PTGB,NaN,"SP5179-Lineage-Edwardsville, KS",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
45,4300582209,NaN,NaN,Smithfield Foods,NaN,NaN,PTGB,NaN,SU0000002228-THE DELI SOURCE INC,Pickup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
46,4300572971,NaN,NaN,Smithfield Foods,NaN,NaN,RYDE,NaN,"SP1232-SFD-Sioux Falls, SD",Delivery,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO


In [26]:
display(df['Truck Number'])

39       NaN
40       NaN
44       NaN
45       NaN
46       NaN
47       NaN
48       NaN
49       NaN
50       NaN
51       NaN
52       NaN
53       NaN
54       NaN
55       NaN
56       NaN
57       NaN
58       NaN
59       NaN
60       NaN
61       NaN
71       NaN
72       NaN
79       NaN
80       NaN
81       NaN
82       NaN
83       NaN
84       NaN
119      NaN
120      NaN
        ... 
3641     NaN
3642     NaN
3643     NaN
3644     NaN
3651     NaN
3652     NaN
3872     NaN
3873     NaN
4821     NaN
4822     NaN
5281     NaN
5282     NaN
5421     NaN
5422     NaN
5424     NaN
5425     NaN
5429     NaN
5430     NaN
5572     NaN
5573     NaN
5809     NaN
5810     NaN
5811     NaN
5812     NaN
5817     NaN
5818     NaN
8701     NaN
8702     NaN
21223    NaN
21224    NaN
Name: Truck Number, Length: 168, dtype: object

In [27]:
df['City']

39             Burleson
40               Cudahy
44         Edwardsville
45              Antioch
46          Sioux Falls
47                Omaha
48         Edwardsville
49              Antioch
50           Greenfield
51           Sioux City
52         Indianapolis
53           Sioux City
54             Tar Heel
55           Sioux City
56           Greenfield
57           Sioux City
58         Edwardsville
59           Sioux City
60         Edwardsville
61           Sioux City
71          Orange City
72                Milan
79        Saint Charles
80              Denison
81          Saint James
82              Antioch
83          Orange City
84                Milan
119         Sioux Falls
120         Sioux Falls
              ...      
3641            Seattle
3642            Denison
3643            Seattle
3644            Denison
3651            Chicago
3652             Cudahy
3872          Macclenny
3873            Sanford
4821       Winter Haven
4822            Sanford
5281          Ri

In [28]:
for f in df['City']:
    s = str(f).lower()
    df.loc[df['City'] == f, 'City'] = s
for f in fd['City']:
    s = str(f).lower()
    fd.loc[fd['City'] == f, 'City'] = s

In [29]:
df.loc[:, 'City'] = df.loc[:, 'City'] + ',' + df.loc[:, 'State']
fd.loc[:, 'City'] = fd.loc[:, 'City'] + ',' + fd.loc[:, 'State']

In [30]:
display(fd['City'])

45              antioch,IL
47                omaha,NE
49              antioch,IL
51           sioux city,IA
53           sioux city,IA
55           sioux city,IA
57           sioux city,IA
58         edwardsville,KS
59           sioux city,IA
60         edwardsville,KS
61           sioux city,IA
72                milan,MO
79        saint charles,IL
80              denison,IA
81          saint james,MN
82              antioch,IL
83          orange city,IA
84                milan,MO
119         sioux falls,SD
120         sioux falls,SD
142              ankeny,IA
143         cedar falls,IA
157             norfolk,VA
158             norfolk,VA
162     monroe township,NJ
163     monroe township,NJ
201          smithfield,VA
202            tar heel,NC
203          smithfield,VA
204            tar heel,NC
               ...        
2537            le mars,IA
2539            le mars,IA
2544            le mars,IA
2546            le mars,IA
2548            le mars,IA
2580            sanford,NC
2

In [31]:

df2 = pd.read_csv('./Downloads/cities.csv')
df2.drop_duplicates(subset = ['city'], keep = 'last', inplace = True)

class Pickup:
    def __init__(self):
        pass
class Delivery:
    def __init__(self):
        pass

class Load:
    def __init__(self):
        self.pickup = Pickup()
        self.delivery = Delivery()

class Initial_Position:
    def __init__(self):
        pass
class Truck:
    def __init__(self):
        self.Initial_Position = Initial_Position()
L = []
for f, o in (df.groupby('Load Number')['City']):
    o = o.values
    load = Load()
    L.append(load)
    load.Id = f
    load.pickup.Latitude = df2.loc[(df2['city'] == o[1]), 'latitude'].values[0]
    load.pickup.Longitude = df2.loc[(df2['city'] == o[1]), 'longitude'].values[0]
    load.delivery.Latitude = df2.loc[(df2['city'] == o[0]), 'latitude'].values[0]
    load.delivery.Longitude = df2.loc[(df2['city'] == o[0]), 'longitude'].values[0]
    StrtDate = (df.loc[(df['City'] == o[1]) & (df['Load Number'] == load.Id), 'Appointment Time'].dt.date)
    FinDate = df.loc[(df['City'] == o[0]) & (df['Load Number'] == load.Id), 'Appointment Time'].dt.date
    StrtDateTime = pd.to_datetime(StrtDate, format = '%Y-%m-%d %H:%M:%S')
    FinDateTime = pd.to_datetime(FinDate, format = '%Y-%m-%d %H:%M:%S')
    load.pickup.EarlyAppointmentTime = str(StrtDateTime.values[0]).split('T')
    load.pickup.EarlyAppointmentTime = str(load.pickup.EarlyAppointmentTime[0])+' '+str((str(load.pickup.EarlyAppointmentTime[1]).split('.'))[0])
    load.pickup.EarlyAppointmentTime = datetime.strptime(load.pickup.EarlyAppointmentTime[:18], '%Y-%m-%d %H:%M:%S')
    #print(load.pickup.EarlyAppointmentTime)
    load.delivery.EarlyAppointmentTime = str(FinDateTime.values[0]).split('T')
    load.delivery.EarlyAppointmentTime = str(load.delivery.EarlyAppointmentTime[0])+' '+str((str(load.delivery.EarlyAppointmentTime[1]).split('.'))[0])
    load.delivery.EarlyAppointmentTime = datetime.strptime(load.delivery.EarlyAppointmentTime[:18], '%Y-%m-%d %H:%M:%S')
    #print(load.pickup.EarlyAppointmentTime, load.delivery.EarlyAppointmentTime)
    load.pickup.LatestAppointmentTime = load.pickup.EarlyAppointmentTime + timedelta(hours = 23, minutes = 59, seconds = 59) 
    load.delivery.LatestAppointmentTime = load.delivery.EarlyAppointmentTime + timedelta(hours = 23, minutes = 59, seconds = 59)
    #print(load.pickup.LatestAppointmentTime, load.delivery.LatestAppointmentTime)
    #print(load.pickup.EarlyAppointmentTime, load.pickup.LatestAppointmentTime, load.delivery.EarlyAppointmentTime, load.delivery.LatestAppointmentTime)
#     break


In [32]:
for l in L:
    print(l.pickup.Latitude)

40.202239990234375
36.456390380859375
36.456390380859375
43.54730224609375
35.72126770019531
40.91142654418945
42.959739685058594
42.024986267089844
42.024986267089844
42.024986267089844
42.47724151611328
43.54730224609375
39.78504180908203
42.024986267089844
41.25653839111328
42.492225646972656
42.492225646972656
35.47987747192383
37.227928161621094
42.496341705322266
42.496341705322266
42.496341705322266
42.496341705322266
42.496341705322266
42.959739685058594
36.456390380859375
39.79529571533203
42.496341705322266
40.62521743774414
33.448375701904304
32.91100311279297
46.8771858215332
33.51858901977539
43.07979965209961
45.40761947631836
26.77534103393555
28.039464950561523
29.187198638916016
34.49426651000977
32.7359619140625
40.62521743774414
36.32958984375
43.00674057006836
35.47987747192383
35.47987747192383
35.47987747192383
34.74008178710937
32.91100311279297
41.13998031616211
42.79415512084961
39.78504180908203
35.47987747192383
41.58683395385742
39.739234924316406
43.0072097

In [33]:
for load in L:
    url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
        load.pickup.Latitude, load.pickup.Longitude, load.delivery.Latitude, load.delivery.Longitude)
    print(load.pickup.Latitude, load.pickup.Longitude, load.delivery.Latitude, load.delivery.Longitude)
    info = requests.get(url).json()['response']['route'][0]['leg'][0]
    load.Distance = info['length']
    load.TimeTaken = info['travelTime']

40.202239990234375 -93.12520599365234 43.00720977783203 -96.058349609375
36.456390380859375 -76.8566665649414 27.50356101989746 -99.50755310058594
36.456390380859375 -76.8566665649414 27.50356101989746 -99.50755310058594
43.54730224609375 -96.72833251953124 27.50356101989746 -99.50755310058594
35.72126770019531 -77.91554260253906 36.98237228393555 -76.63106536865234
40.91142654418945 -90.64735412597656 27.50356101989746 -99.50755310058594
42.959739685058594 -87.8614730834961 41.87811279296875 -87.62979888916016
42.024986267089844 -95.35888671875 47.60620880126953 -122.33206939697266
42.024986267089844 -95.35888671875 47.60620880126953 -122.33206939697266
42.024986267089844 -95.35888671875 47.60620880126953 -122.33206939697266
42.47724151611328 -88.09564208984375 43.98207473754883 -94.6283187866211
43.54730224609375 -96.72833251953124 26.10035514831543 -98.26306915283205
39.78504180908203 -85.76942443847656 26.10035514831543 -98.26306915283205
42.024986267089844 -95.35888671875 41.91419

In [34]:
for l in L:
    print((l.Distance), (l.TimeTaken))

625355 27025
2739880 112061
2739880 112061
2023667 81527
248395 12607
2114807 85230
144913 9287
2790583 106622
2790583 106622
2790583 106622
696073 29328
2233988 90658
2413487 95403
674974 29668
292197 11907
584847 26566
1054331 45326
743394 31105
53244 2503
446119 17679
446119 17679
1135417 45239
2341639 96266
1085783 43344
1749108 69587
1103440 44830
619673 25388
1135417 45239
2404389 89844
34 4
590275 25528
131020 5796
225086 11773
816803 33451
7277 753
32 4
93352 5033
310741 12714
577805 24467
126922 6646
1446107 53746
286669 12498
558512 24540
942095 40634
147527 6797
442839 18815
1003119 38754
590275 25528
1534356 56837
1444504 58632
40509 2536
781792 32939
537278 22202
1104206 47691
645152 26803
1000503 42383
1104206 47691
1104206 47691
0 0
128929 7342
38393 3514
1103440 44830
297920 14124
0 0
10 2
394616 16955
625355 27025
186482 8185
993364 37516
2636965 101191
2522179 93635
1819354 67849
1630962 65711
1547061 61861
1476176 60553
2522179 93635
0 0
1000503 42383
77803 4777
7780

In [35]:
for f in L:
    s = datetime.fromtimestamp(f.TimeTaken)
    l = datetime.fromtimestamp(0)
    f.TimeTaken = (s-l)*21.0/11 + timedelta(hours = 1)

In [36]:
#Id, Inital_Position, Final_Position, Possible, PickupTime, EAT
T = []
for f, o in fd.groupby('Truck Number')['City']:
    truck = Truck()
    T.append(truck)
    o = o.values
    truck.Id = f
    if(len(o) == 2):
        truck.Initial_Position.Latitude = df2.loc[(df2['city'] == o[1]), 'latitude'].values[0]
        truck.Initial_Position.Longitude = df2.loc[(df2['city'] == o[1]), 'longitude'].values[0]
        truck.PickupTime = fd.loc[(fd['Truck Number'] == truck.Id) & (fd['City'] == o[1]), 'Appointment Time'].values[0]
        truck.PickupTime = datetime.strptime(str(truck.PickupTime)[:18], '%Y-%m-%d %H:%M:%S')
    else:
        truck.Initial_Position.Latitude = df2.loc[(df2['city'] == o[0]), 'latitude'].values[0]
        truck.Initial_Position.Longitude = df2.loc[(df2['city'] == o[0]), 'longitude'].values[0]
        truck.PickupTime = fd.loc[(fd['Truck Number'] == truck.Id) & (fd['City'] == o[0]), 'Appointment Time'].values[0]
        truck.PickupTime = datetime.strptime(str(truck.PickupTime)[:18], '%Y-%m-%d %H:%M:%S')
    print(truck.PickupTime)


2019-05-31 21:30:00
2019-05-31 15:30:00
2019-05-31 21:30:00
2019-05-31 18:00:00
2019-05-31 18:00:00
2019-05-31 18:00:00
2019-05-31 18:00:00
2019-05-31 18:00:00
2019-05-31 18:00:00
2019-05-31 11:00:00
2019-05-31 10:30:00
2019-05-31 10:30:00
2019-05-31 11:00:00
2019-05-31 14:00:00
2019-05-31 01:30:00
2019-05-31 21:00:00
2019-05-31 04:30:00
2019-05-31 05:00:00
2019-05-31 04:00:00
2019-05-31 14:30:00
2019-05-31 04:00:00
2019-05-31 02:00:00
2019-05-31 10:00:00
2019-05-31 05:00:00
2019-05-31 22:00:00
2019-05-31 23:30:00
2019-05-31 18:00:00
2019-05-31 18:00:00
2019-05-31 22:00:00
2019-05-31 17:30:00
2019-05-31 15:30:00
2019-05-31 02:00:00
2019-05-31 12:00:00
2019-05-31 06:30:00
2019-05-31 13:00:00
2019-05-31 09:30:00
2019-05-31 00:30:00
2019-05-31 04:30:00
2019-05-31 13:30:00
2019-05-31 10:00:00
2019-05-31 07:16:00
2019-05-31 03:31:00
2019-05-31 07:00:00
2019-05-31 11:30:00
2019-05-31 05:00:00
2019-05-31 01:26:00
2019-05-31 05:01:00
2019-05-31 14:00:00
2019-05-31 12:00:00
2019-05-31 13:00:00


In [ ]:
%%time
# Preference, Distance, EAT, Possible, IndexNumber
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None
        self.prev = None
def Preference_Truck_Load(a, b):
    if a > b:
        return 1
    elif a < b and a > b - timedelta(days = 1):
        return 2
    elif a < b - timedelta(days = 1) and a > b - timedelta(days = 2):
        return 3
    elif a < b - timedelta(days = 2) and a > b - timedelta(days = 3):
        return 4
    elif a < b - timedelta(days = 3) and a > b - timedelta(days = 4):
        return 5
    elif a < b - timedelta(days = 4) and a > b - timedelta(days = 5):
        return 6
    else:
        return 7
class Truck_Load_Assignment:
    def __init__(self):
        pass
    
    def __lt__(self, other):
#         print("Called")
        if(self.Preference != other.Preference):
            return self.Preference < other.Preference
        return self.Dead_Head > other.Dead_Head
TLA = [[] for _ in range(len(T))]
for t in range(len(T)):
    Index = 1
    for l in L:
        truck_load_assignment = Truck_Load_Assignment()
        url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
            T[t].Initial_Position.Latitude, T[t].Initial_Position.Longitude, l.pickup.Latitude, l.pickup.Longitude)
        apiReturn = requests.get(url).json()
        info = apiReturn['response']['route'][0]['leg'][0]
        distance = info['length']
        time = info['travelTime']
        s = datetime.fromtimestamp(time)
        l1 = datetime.fromtimestamp(0)
        s = s-l1
        s = T[t].PickupTime + s*21.0/11
        if s < l.pickup.LatestAppointmentTime:
            truck_load_assignment.Possible = True
            truck_load_assignment.Preference = Preference_Truck_Load(s, l.pickup.EarlyAppointmentTime)
            truck_load_assignment.Dead_Head = distance
            truck_load_assignment.E_Pickup_T = s
            truck_load_assignment.E_Delivery_T = s+l.TimeTaken
            truck_load_assignment.TimeTaken = l.TimeTaken
            truck_load_assignment.Index = Index
            truck_load_assignment.No_of_Loads = 1
            truck_load_assignment.head = Node(Index)
            truck_load_assignment.other = Node(0)
            truck_load_assignment.tother = truck_load_assignment.other
            truck_load_assignment.tail = truck_load_assignment.head
            truck_load_assignment.Final_Latitude = l.delivery.Latitude
            truck_load_assignment.Final_Longitude = l.delivery.Longitude
            truck_load_assignment.Initial_Latitude = l.pickup.Latitude
            truck_load_assignment.Initial_Longitude = l.pickup.Longitude
            truck_load_assignment.LatestAppointmentTime = l.delivery.LatestAppointmentTime
            truck_load_assignment.EarlyAppointmentTime = l.delivery.EarlyAppointmentTime
            truck_load_assignment.Previous_In = -1
        else:
            truck_load_assignment.Possible = False
            truck_load_assignment.Index = Index
            truck_load_assignment.head = Node(Index)
            truck_load_assignment.tail = truck_load_assignment.head
            truck_load_assignment.No_of_Loads = 0
            truck_load_assignment.Preference = 8
            truck_load_assignment.Previous_In = -1
            truck_load_assignment.other = Node(0)
            truck_load_assignment.tother = truck_load_assignment.other
            truck_load_assignment.Dead_Head = sys.maxsize
        TLA[t].append(truck_load_assignment)
        Index = Index + 1
    TLA[t] = sorted(TLA[t])
for i in range(len(T)):
    for j in range(len(L)):
        print(TLA[i][j].Preference, TLA[i][j].Dead_Head)

Travel_T = [[] for _ in range(len(L))]
Dead = [[] for _ in range(len(L))]
for i in range(len(L)):
    for j in range(len(L)):
        if(i == j):
            Dead[i].append(0)
            Travel_T.append(0)
        else:
            url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
                 L[i].delivery.Latitude, L[i].delivery.Longitude, L[j].pickup.Latitude, L[j].pickup.Longitude)
            api_return = requests.get(url).json()
            info = api_return['response']['route'][0]['leg'][0]
            Dead[i].append(info['length'])
            s = datetime.fromtimestamp(info['travelTime'])
            l1 = datetime.fromtimestamp(0)
            s = (s-l1)*21.0/11 + timedelta(hours = 1)
            Travel_T[i].append(s)
        

In [ ]:
# r = TLA[0][2].head
# while(r != None):
#     print(r.data)
#     r = r.next

In [ ]:
%%time
def MAX(a, b):
    if a > b:
        return a
    return b

for i in range(len(T)):
    for j in range(1, len(L)):
        x = TLA[i][j].No_of_Loads
        Dead_Head = 0
        E_Pickup_T = datetime.fromtimestamp(0)
        E_Delivery_T = datetime.fromtimestamp(0)
        if(TLA[i][j].Preference < 8):
            In = -1
            for k in range(j):
                if TLA[i][k].Preference < TLA[i][j].Preference:
                    y = TLA[i][j].No_of_Loads
                    if(TLA[i][k].E_Delivery_T + Travel_T[k][j] + TLA[i][j].TimeTaken <= TLA[i][j].LatestAppointmentTime):
                        l = MAX(x, TLA[i][k].No_of_Loads + y)
                        if(TLA[i][k].No_of_Loads + y == x):
                            if(Dead[k][j] + TLA[i][k].Dead_Head < Dead_Head):
                                Dead_Head = TLA[i][k].Dead_Head + Dead[k][j]
                                In = k
                                E_Pickup_T = TLA[i][k].E_Delivery_T + Travel_T[k][j]
                                E_Delivery_T = E_Pickup_T + TLA[i][j].TimeTaken
                        elif(TLA[i][k].No_of_Loads + y > x):
                            Dead_Head = Dead[k][j] + TLA[i][k].Dead_Head
                            In = k
                            E_Pickup_T = TLA[i][k].E_Delivery_T + Travel_T[k][j]
                            E_Delivery_T = E_Pickup_T + TLA[i][j].TimeTaken
                        x = l
            if In > -1:
                q = TLA[i][In].head
                while(q != None):
                    TLA[i][j].tail.next = Node(q.data)
                    TLA[i][j].tail.next.prev = TLA[i][j].tail
                    TLA[i][j].tail = TLA[i][j].tail.next
                    q = q.next
                TLA[i][j].Dead_Head = Dead_Head
                TLA[i][j].E_Pickup_T = E_Pickup_T
                TLA[i][j].E_Delivery_T = E_Delivery_T
        TLA[i][j].No_of_Loads = x


                
TD = [[] for _ in range(len(L))]
for i in range(len(T)):
    for j in range(len(L)):
        url = "https://route.api.here.com/routing/7.2/calculateroute.json?\
app_id=SWBYU0csWZrpTY9aQBqL&app_code=GlpK2MA4IGFM2MsPX3ICPg&gen=9&mode=\
fastest%3Btruck%3Btraffic%3Aenabled&legattributes=none%2Cle%2Ctt&\
resolution=10&routeattributes=none%2Clg&departure=2019-06-04T21%3A38%\
3A02%2B00%3A00&waypoint0=geo%21{}%2C{}&waypoint1=geo%21{}%2C{}".format(
                 T[i].Initial_Position.Latitude, T[i].Initial_Position.Longitude, L[j].pickup.Latitude, L[j].pickup.Longitude)
        api_return = requests.get(url).json()
        info = api_return['response']['route'][0]['leg'][0]
        TD[i].append(info['length'])            
        

In [ ]:
for i in range(len(T)):
    for j in range(len(L)):
        if(TLA[i][j].Preference < 8):
            current = TLA[i][j].head
            while(current != None):
                print(current.data)
                current = current.next
            print("------__________------- ")
    print(" ")

In [ ]:
for i in range(len(T)):
    for j in range(len(L)):
        if(TLA[i][j].Preference < 8):
            temp = None
            current = TLA[i][j].head
            while(current != None):
                temp = current.prev
                current.prev = current.next
                current.next = temp
                current = current.prev
            if(temp != None):
                TLA[i][j].head = temp.prev
            current = TLA[i][j].head
            while(current != None):
                print(current.data)
                current = current.next
            print("------__________------- ")
    print(" ")
        
            

In [ ]:
for i in range(1, 2):
    Distance = [[] for _ in range(len(L))]
    for j in range(len(L)):
        for k in range(len(L)):
            Distance[j].append(TLA[i][j].Dead_Head+TLA[i-1][k].Dead_Head)
for i in range(1, 2):
    for j in range(len(L)):
        if(TLA[i][j].Preference < 8):
            sum = 0
            x = TLA[i][j].No_of_Loads
            Dead_Head = TLA[i][j].Dead_Head
            Previous_In = -1
            for k in range(len(L)):
                if(TLA[i][k].Preference < 8):
                    y = TLA[i-1][k].No_of_Loads
                    p = TLA[i][j].head
                    while(p != None):
                        q = TLA[i-1][k].head
                        while(q != None):
                            if(p.data == q.data):
                                if(p == TLA[i][j].head):
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TLA[i-1][k].Dead_Head < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                            else:
                                                if(TD[i-1][q.next.data-1]+TLA[i][j].Dead_Head < TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[q.prev.data-1][q.next.data-1]+TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+TLA[i-1][k].Dead_Head < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                            else:
                                                if(TD[i-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                else:
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.data-1] < TLA[i-1][k].Dead_Head):
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                            else:
                                                if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] < Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1] + TLA[i-1][k].Dead_Head < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i-1][k].Dead_Head
                                            else:
                                                if(TD[i-1][q.next.data-1] < Dead[p.prev.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                sum += 1
                                break
                            q = q.next
                        p = p.next
                    l = MAX(x, TLA[i][j].No_of_Loads+y-sum)
                    if(TLA[i][j].No_of_Loads+y-sum == x):
                        if(Dead_Head > Distance[j][k]):
                            Previous_In = k
                            Dead_Head = Distance[j][k]
                    elif(TLA[i][j].No_of_Loads+y-sum > x):
                        Previous_In = k
                        Dead_Head = Distance[j][k]
                    x = l
            if(sum > 0):
                p = TLA[i][j].head
                while(p != None):
                    q = TLA[i-1][Previous_In].head
                    while(q != None):
                        if(p.data == q.data):
                            if(p == TLA[i][j].head):
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TLA[i][j].Dead_Head > TLA[i-1][Previous_In].Dead_Head):
                                                TLA[i][j].head = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < TLA[i][j].Dead_Head+TD[i-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                TLA[i][j].head = None
                                        else:
                                            if(TLA[i][j].Dead_Head+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1] + TLA[i-1][Previous_In].Dead_Head < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > TD[i][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(TD[i][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                            else:
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.data-1] > TLA[i-1][Previous_In].Dead_Head):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1] + TLA[i-1][Previous_In].Dead_Head < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > Dead[p.prev.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                            break
                        q = q.next
                    p = p.next
            q = TLA[i-1][Previous_In].head
            while(q != None):
                p = TLA[i][j].head
                o = 1
                while(p != None):
                    if(p.data == q.data):
                        o = 0
                        break
                    p = p.next
                if(o == 1):
                    if(TLA[i][j].other.data == 0):
                        TLA[i][j].other = Node(q.data)
                        TLA[i][j].tother = TLA[i][j].other
                    else:
                        TLA[i][j].tother.next = Node(q.data)
                        TLA[i][j].tother.next.prev = TLA[i][j].tother
                        TLA[i][j].tother = TLA[i][j].tother.next
                q = q.next
            TLA[i][j].No_of_Loads = x
            TLA[i][j].Dead_Head = Dead_Head
            TLA[i][j].Previous_In = Previous_In
        print(TLA[i][j].Preference, TLA[i][j].No_of_Loads, TLA[i][j].Dead_Head, TLA[i][j].Previous_In)


In [ ]:
# r = TLA[1][0].head
# while(r != None):
#     print(r.data)
#     r = r.next

In [ ]:
for i in range(2, len(T)):
    for j in range(len(L)):
        Distance = [[] for _ in range(len(L))]
        if(TLA[i][j].Preference < 8):
            sum = 0
            x = TLA[i][j].No_of_Loads
            Dead_Head = TLA[i][j].Dead_Head
            Previous_In = -1
            for k in range(len(L)):
                Distance[j].append(TLA[i][j].Dead_Head+TLA[i-1][k].Dead_Head)
                if(TLA[i][k].Preference < 8):
                    p = TLA[i][j].head
                    y = TLA[i-1][k].No_of_Loads
                    while(p != None):
                        q = TLA[i-1][k].head
                        while(q != None):
                            if(p.data == q.data):
                                if(p == TLA[i][j].head):
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TD[i-1][q.data-1] < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                            else:
                                                if(TD[i-1][q.next.data-1]+TLA[i][j].Dead_Head < TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[q.prev.data-1][q.next.data-1]+TLA[i][j].Dead_Head):
                                                    Distance[j][k] -= TLA[i][j].Dead_Head
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+TD[i-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                            else:
                                                if(TD[i-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[q.prev.data-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                else:
                                    if(p.next == None):
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.data-1] < TD[i-1][q.data-1]):
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                            else:
                                                if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] < Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                    else:
                                        if(q == TLA[i-1][k].head):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1] + TD[i-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= TD[i-1][q.data-1]
                                            else:
                                                if(TD[i-1][q.next.data-1] < Dead[p.prev.data-1][p.next.data-1]):
                                                    Distance[j][k] += (TD[i-1][q.next.data-1]-TD[i-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                        else:
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                            else:
                                                if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                else:
                                                    Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                
                                sum += 1
                                break
                            q = q.next
                        p = p.next
                    a = 1
                    Previous = k
                    while(TLA[i-a][Previous].Previous_In > -1):
                        p = TLA[i][j].head
                        while(p != None):
                            q = TLA[i-a][Previous].other
                            while(q != None):
                                if(p.data == q.data):
                                    if(p == TLA[i][j].head):
                                        if(p.next == None):
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(TD[i-a-1][q.data-1] < TLA[i][j].Dead_Head):
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                                    else:
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                else:
                                                    if(TD[i-a-1][q.next.data-1]+TLA[i][j].Dead_Head < TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                            else:
                                                if(q.next == None):
                                                    if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[q.prev.data-1][q.next.data-1]+TLA[i][j].Dead_Head):
                                                        Distance[j][k] -= TLA[i][j].Dead_Head
                                                    else:
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                        else:
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(TD[i][p.next.data-1]+TD[i-a-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                else:
                                                    if(TD[i-a-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                            else:
                                                if(q.next == None):
                                                    if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[q.prev.data-1][q.next.data-1] < TD[i][p.next.data-1]):
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] += (TD[i][p.next.data-1]-TD[i][p.data-1]-Dead[p.data-1][p.next.data-1])
                                    else:
                                        if(p.next == None):
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(Dead[p.prev.data-1][p.data-1] < TD[i-a-1][q.data-1]):
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                    else:
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                else:
                                                    if(TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-a-1][q.next.data-1]):
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                    else:
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                            else:
                                                if(q.next == None):
                                                    if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] < Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] -= Dead[p.prev.data-1][p.data-1]
                                        else:
                                            if(q == TLA[i-a][Previous].other):
                                                if(q.next == None):
                                                    if(Dead[p.prev.data-1][p.next.data-1] + TD[i-a-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= TD[i-a-1][q.data-1]
                                                else:
                                                    if(TD[i-a-1][q.next.data-1] < Dead[p.prev.data-1][p.next.data-1]):
                                                        Distance[j][k] += (TD[i-a-1][q.next.data-1]-TD[i-a-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                                    else:
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                            else:
                                                if(q.next == None):
                                                    if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] -= Dead[q.prev.data-1][q.data-1]
                                                else:
                                                    if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                        Distance[j][k] += (Dead[p.prev.data-1][p.next.data-1]-Dead[p.prev.data-1][p.data-1]-Dead[p.data-1][p.next.data-1])
                                                    else:
                                                        Distance[j][k] += (Dead[q.prev.data-1][q.next.data-1]-Dead[q.prev.data-1][q.data-1]-Dead[q.data-1][q.next.data-1])
                                    sum += 1
                                    break
                                q = q.next
                            p = p.next
                        Previous = TLA[i-a][Previous].Previous_In
                        a += 1
                    l = MAX(x, TLA[i][j].No_of_Loads+y-sum)
                    if(TLA[i][j].No_of_Loads+y-sum == x):
                        if(Dead_Head > Distance[j][k]):
                            Previous_In = k
                            Dead_Head = Distance[j][k]
                    elif(TLA[i][j].No_of_Loads+y-sum > x):
                        Previous_In = k
                        Dead_Head = Distance[j][k]
                    x = l
            if(sum > 0):
                p = TLA[i][j].head
                while(p != None):
                    q = TLA[i-1][Previous_In].head
                    while(q != None):
                        if(p.data == q.data):
                            if(p == TLA[i][j].head):
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TLA[i][j].Dead_Head > TD[i-1][q.data-1]):
                                                TLA[i][j].head = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < TLA[i][j].Dead_Head+TD[i-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < TLA[i][j].Dead_Head):
                                                TLA[i][j].head = None
                                        else:
                                            if(TLA[i][j].Dead_Head+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                TLA[i][j].head = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1] + TD[i-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > TD[i][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(TD[i][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                TLA[i][j].head = p.next
                                                p.next.prev = None
                                                p.prev = None
                                                p.next = None
                            else:
                                if(p.next == None):
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.data-1] > TD[i-1][q.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                        else:
                                            if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                p.prev.next = None
                                                p.prev = None
                                else:
                                    if(q == TLA[i-1][Previous_In].head):
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1] + TD[i-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.next = None
                                                p.prev = None
                                        else:
                                            if(TD[i-1][q.next.data-1] > Dead[p.prev.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                    else:
                                        if(q.next == None):
                                            if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                                        else:
                                            if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                p.prev.next = p.next
                                                p.next.prev = p.prev
                                                p.prev = None
                                                p.next = None
                            break
                        q = q.next
                    p = p.next
            if(sum > 0):
                a = 1
                Previous = Previous_In
                while(TLA[i-a][Previous].Previous_In > -1):
                    p = TLA[i][j].head
                    while(p != None):
                        q = TLA[i-a][Previous].other
                        while(q != None):
                            if(p.data == q.data):
                                if(p == TLA[i][j].head):
                                    if(p.next == None):
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(TD[i][p.data-1] > TD[i-a-1][q.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1] < TD[i][p.data-1]+TD[i-a-1][q.next.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(TD[i][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    TLA[i][j].head = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.next = None
                                                    q.prev = None
                                    else:
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1] + TD[i-a-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.next.data-1] > TD[i][p.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(TD[i][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < TD[i][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(TD[i][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    TLA[i][j].head = p.next
                                                    p.next.prev = None
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.prev = None
                                                    q.next = None
                                else:
                                    if(p.next == None):
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.data-1] > TD[i-a-1][q.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.data-1]+Dead[q.data-1][q.next.data-1] < Dead[p.prev.data-1][p.data-1]+TD[i-a-1][q.next.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(Dead[p.prev.data-1][p.data-1]+Dead[q.prev.data-1][q.next.data-1] > Dead[q.prev.data-1][q.data-1]+Dead[q.data-1][q.next.data-1]):
                                                    p.prev.next = None
                                                    p.prev = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.prev = None
                                                    q.next = None
                                    else:
                                        if(q == TLA[i-a][Previous].other):
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1] + TD[i-a-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.next = None
                                                    p.prev = None
                                                else:
                                                    TLA[i-a][Previous].other = None
                                            else:
                                                if(TD[i-a-1][q.next.data-1] > Dead[p.prev.data-1][p.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    TLA[i-a][Previous].other = q.next
                                                    TLA[i-a][Previous].other.prev = None
                                                    q.next = None
                                        else:
                                            if(q.next == None):
                                                if(Dead[p.prev.data-1][p.next.data-1]+Dead[q.prev.data-1][q.data-1] < Dead[p.prev.data-1][p.data-1]+Dead[p.data-1][p.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.prev.next = None
                                                    q.prev = None
                                            else:
                                                if(Dead[p.prev.data-1][p.next.data-1] < Dead[q.prev.data-1][q.next.data-1]):
                                                    p.prev.next = p.next
                                                    p.next.prev = p.prev
                                                    p.prev = None
                                                    p.next = None
                                                else:
                                                    q.next.prev = q.prev
                                                    q.prev.next = q.next
                                                    q.prev = None
                                                    q.next = None
                                break
                            q = q.next
                        p = p.next
                    Previous = TLA[i-a][Previous].Previous_In
                    a += 1
            p = TLA[i-1][Previous_In].head
            while(p != None):
                q = TLA[i][j].head
                o = 1
                while(q != None):
                    if(p.data == q.data):
                        o = 0
                        break
                    q = q.next
                if(o == 1):
                    if(TLA[i][j].other.data == 0):
                        TLA[i][j].other = Node(p.data)
                        TLA[i][j].tother = TLA[i][j].other
                    else:
                        TLA[i][j].tother.next = Node(p.data)
                        TLA[i][j].tother.next.prev = TLA[i][j].tother
                        TLA[i][j].tother = TLA[i][j].tother.next
                p = p.next
            TLA[i][j].No_of_Loads = x
            TLA[i][j].Dead_Head = Dead_Head
            TLA[i][j].Previous_In = Previous_In
        print(TLA[i][j].Preference, TLA[i][j].No_of_Loads, TLA[i][j].Dead_Head, TLA[i][j].Previous_In)
    print("---------____________---------")


                            
                                
                        
                        
                                
                                
                        
                            
                    
                

In [ ]:
# s = TLA[1][7].other
# while(s != None):
#     print(s.data)
#     s = s.next

In [ ]:
In_dex = 0
x = TLA[len(T)-1][j].No_of_Loads
dead = TLA[len(T)-1][j].Dead_Head
for j in range(len(L)):
    l = MAX(x, TLA[len(T)-1][j].No_of_Loads)
    if(TLA[len(T)-1][j].No_of_Loads == x):
        if(TLA[len(T)-1][j].Dead_Head < dead):
            In_dex = j
            dead = TLA[len(T)-1][j].Dead_Head
    elif(TLA[len(T)-1][j].No_of_Loads > x):
        In_dex = j
        dead = TLA[len(T)-1][j].Dead_Head
    x = l
s = TLA[len(T)-1][In_dex].head
print("Truck:", len(T))
while(s != None):
    print(s.data)
    s = s.next
print("---------____________---------")
i = len(T)-1
Previous = In_dex
while(TLA[i][Previous].Previous_In > -1):
    p = TLA[i][Previous].other
    print("Truck:", i)
    while(p != None):
        print(p.data)
        p = p.next
    print("---------____________---------")
    Previous = TLA[i][Previous].Previous_In
    i -= 1
print("Total Dead_Head:", TLA[len(T)-1][In_dex].Dead_Head)
print("Total No_of_Loads", TLA[len(T)-1][In_dex].No_of_Loads)

In [ ]:
for ind, l in enumerate(L):
    if ind + 1 in [4, 13, 14]:
        print("{}: EarlyPickupApptTime {}, LatestPickupApptTime {}, \n\tDeliveryApptTime {}, latitude {}, longitude {}, dlatitude {}, dlongitude {}".format(
                            ind + 1, l.pickup.EarlyAppointmentTime, l.pickup.LatestAppointmentTime, 
                      l.delivery.EarlyAppointmentTime, l.pickup.Latitude, l.pickup.Longitude, l.delivery.Latitude, l.delivery.Longitude))
    